In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

print("="*80)
print("INSURANCE AGENT PERFORMANCE PREDICTION MODEL")
print("="*80)

# Set random seed for reproducibility
np.random.seed(42)

# Load data
train_data = pd.read_csv('train_storming_round.csv')
test_data = pd.read_csv('test_storming_round.csv')

print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")

#############################################
# PART 1: EXPLORATORY DATA ANALYSIS (EDA)
#############################################
print("\n" + "="*50)
print("EXPLORATORY DATA ANALYSIS")
print("="*50)

# Convert date columns to datetime format for both train and test
date_columns = ['agent_join_month', 'first_policy_sold_month', 'year_month']
for col in date_columns:
    train_data[col] = pd.to_datetime(train_data[col])
    test_data[col] = pd.to_datetime(test_data[col])

# 1. Summary statistics of the dataset
print("\n1. SUMMARY STATISTICS")
print("-"*30)
summary_stats = train_data.describe().T
print(summary_stats)

# 2. Missing values analysis
print("\n2. MISSING VALUES ANALYSIS")
print("-"*30)
missing_train = train_data.isnull().sum()
print("Missing values in training data:")
print(missing_train[missing_train > 0])

# 3. Create target column: if new_policy_count is 0, target = 0, else target = 1
# (Need to create this first for EDA by target)
target = np.where(train_data['new_policy_count'] == 0, 0, 1)
train_data['target'] = target

print("\n3. TARGET DISTRIBUTION")
print("-"*30)
target_counts = train_data['target'].value_counts()
print(f"Target distribution:\n{target_counts}")
print(f"Target percentage:\n{target_counts / len(train_data) * 100}")

# 4. Time Series Analysis - Sales patterns by month
print("\n4. TIME SERIES ANALYSIS")
print("-"*30)
monthly_sales = train_data.groupby('year_month')['new_policy_count'].sum().reset_index()
monthly_avg_sales = train_data.groupby('year_month')['new_policy_count'].mean().reset_index()
monthly_nill_agents = train_data.groupby('year_month')['target'].value_counts().unstack().fillna(0).reset_index()

print("Monthly total sales:")
print(monthly_sales.head())
print("\nMonthly average sales per agent:")
print(monthly_avg_sales.head())
print("\nMonthly NILL vs Non-NILL agents:")
if 0 in monthly_nill_agents.columns and 1 in monthly_nill_agents.columns:
    print(monthly_nill_agents.head())
else:
    print("Could not calculate NILL vs Non-NILL by month due to data structure")

# Plot monthly sales trends
plt.figure(figsize=(12, 6))
plt.plot(monthly_sales['year_month'], monthly_sales['new_policy_count'], marker='o', linestyle='-')
plt.title('Total Policy Sales by Month')
plt.xlabel('Month')
plt.ylabel('Total Policies Sold')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('monthly_sales_trend.png')
plt.close()

# 5. Agent Experience Analysis
print("\n5. AGENT EXPERIENCE ANALYSIS")
print("-"*30)
train_data['agent_experience_months'] = ((train_data['year_month'].dt.year - train_data['agent_join_month'].dt.year) * 12 + 
                                (train_data['year_month'].dt.month - train_data['agent_join_month'].dt.month))

# Calculate months since first policy sold
train_data['months_since_first_policy'] = np.where(pd.notnull(train_data['first_policy_sold_month']),
                                         ((train_data['year_month'].dt.year - train_data['first_policy_sold_month'].dt.year) * 12 + 
                                         (train_data['year_month'].dt.month - train_data['first_policy_sold_month'].dt.month)),
                                         -1)  # -1 for agents who haven't sold a policy yet

print("Agent experience distribution:")
print(train_data['agent_experience_months'].describe())

# Group performance by experience
exp_performance = train_data.groupby('agent_experience_months')['new_policy_count'].agg(['mean', 'median', 'count']).reset_index()
print("\nPerformance by experience months (first 10 rows):")
print(exp_performance.head(10))

# Plot agent performance by experience
plt.figure(figsize=(14, 7))
plt.bar(exp_performance['agent_experience_months'], exp_performance['mean'], alpha=0.7)
plt.title('Average Policy Sales by Agent Experience')
plt.xlabel('Months of Experience')
plt.ylabel('Average Policies Sold')
plt.tight_layout()
plt.savefig('sales_by_experience.png')
plt.close()

# 6. Agent trajectory over time
print("\n6. AGENT TRAJECTORIES")
print("-"*30)
# Select a sample of agents to visualize their trajectories
sample_agents = train_data['agent_code'].unique()[:5]
agent_trajectories = train_data[train_data['agent_code'].isin(sample_agents)]

plt.figure(figsize=(12, 7))
for agent in sample_agents:
    agent_data = train_data[train_data['agent_code'] == agent]
    plt.plot(agent_data['year_month'], agent_data['new_policy_count'], marker='o', label=f'Agent {agent}')

plt.title('Policy Sales Trajectories for Sample Agents')
plt.xlabel('Month')
plt.ylabel('Policies Sold')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('agent_trajectories.png')
plt.close()

# 7. Correlation Analysis
print("\n7. CORRELATION ANALYSIS")
print("-"*30)
numeric_cols = train_data.select_dtypes(include=['float64', 'int64']).columns
correlation = train_data[numeric_cols].corr()

# Print correlations with target
target_corr = correlation['target'].sort_values(ascending=False)
print("Features correlation with target (top 10):")
print(target_corr.head(10))

# Plot correlation heatmap of important features
plt.figure(figsize=(14, 12))
top_features = target_corr.index[:15]  # Top 15 correlated features
sns.heatmap(train_data[top_features].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of Top Features')
plt.tight_layout()
plt.savefig('correlation_heatmap.png')
plt.close()

# 8. Feature distributions by target
print("\n8. FEATURE DISTRIBUTIONS BY TARGET")
print("-"*30)
important_features = ['new_policy_count', 'unique_customers', 'unique_quotations', 
                      'unique_proposal', 'agent_experience_months', 'agent_age']

for feature in important_features[:3]:  # Show first 3 for brevity
    plt.figure(figsize=(10, 6))
    sns.histplot(data=train_data, x=feature, hue='target', bins=30, kde=True, element='step')
    plt.title(f'Distribution of {feature} by Target')
    plt.tight_layout()
    plt.savefig(f'{feature}_by_target.png')
    plt.close()
    
    print(f"Summary of {feature} by target:")
    print(train_data.groupby('target')[feature].describe())

# 9. Funnel Analysis
print("\n9. FUNNEL ANALYSIS")
print("-"*30)
funnel_avg = train_data[['unique_proposal', 'unique_quotations', 'unique_customers', 'new_policy_count']].mean()
print("Average funnel metrics:")
print(funnel_avg)

# Calculate conversion rates
train_data['proposal_to_quotation_rate'] = train_data['unique_quotations'] / train_data['unique_proposal'].replace(0, np.nan)
train_data['quotation_to_customer_rate'] = train_data['unique_customers'] / train_data['unique_quotations'].replace(0, np.nan)
train_data['customer_to_policy_rate'] = train_data['new_policy_count'] / train_data['unique_customers'].replace(0, np.nan)

print("\nConversion rates by target:")
conversion_by_target = train_data.groupby('target')[['proposal_to_quotation_rate', 'quotation_to_customer_rate', 'customer_to_policy_rate']].mean()
print(conversion_by_target)

# 10. Recent Activity vs. Overall Activity
print("\n10. RECENT VS OVERALL ACTIVITY")
print("-"*30)
recent_cols = ['unique_proposals_last_7_days', 'unique_quotations_last_7_days', 'unique_customers_last_7_days']
overall_cols = ['unique_proposal', 'unique_quotations', 'unique_customers']

for recent, overall in zip(recent_cols, overall_cols):
    train_data[f'{recent}_ratio'] = train_data[recent] / train_data[overall].replace(0, np.nan)
    print(f"Average {recent}_ratio: {train_data[f'{recent}_ratio'].mean()}")

print("\nRecent activity ratios by target:")
recent_ratio_cols = [col for col in train_data.columns if '_ratio' in col]
print(train_data.groupby('target')[recent_ratio_cols].mean())

# 11. Agent Segmentation by Performance
print("\n11. AGENT PERFORMANCE SEGMENTATION")
print("-"*30)
# Create a new column for agent performance category
agent_avg_performance = train_data.groupby('agent_code')['new_policy_count'].mean()
performance_quantiles = agent_avg_performance.quantile([0.33, 0.67])

low_performers = agent_avg_performance[agent_avg_performance <= performance_quantiles[0.33]].index
medium_performers = agent_avg_performance[(agent_avg_performance > performance_quantiles[0.33]) & 
                                         (agent_avg_performance <= performance_quantiles[0.67])].index
high_performers = agent_avg_performance[agent_avg_performance > performance_quantiles[0.67]].index

print(f"Number of low performers: {len(low_performers)}")
print(f"Number of medium performers: {len(medium_performers)}")
print(f"Number of high performers: {len(high_performers)}")

# Add performance category to the dataset
train_data['performance_category'] = 'medium'
train_data.loc[train_data['agent_code'].isin(low_performers), 'performance_category'] = 'low'
train_data.loc[train_data['agent_code'].isin(high_performers), 'performance_category'] = 'high'

# Calculate NILL rate by performance category
nill_by_category = train_data.groupby('performance_category')['target'].mean()
print("\nProbability of non-NILL by performance category:")
print(nill_by_category)

#############################################
# PART 2: FEATURE ENGINEERING
#############################################
print("\n" + "="*50)
print("FEATURE ENGINEERING")
print("="*50)

def create_target(df):
    """Create target column: if new_policy_count is 0, target = 0, else target = 1"""
    if 'new_policy_count' in df.columns:
        return np.where(df['new_policy_count'] == 0, 0, 1)
    return None

# Create target for training data only
target = create_target(train_data)

def feature_engineering(df, is_training=True):
    """Comprehensive feature engineering function"""
    # Make a copy to avoid modifying the original
    processed_df = df.copy()
    
    # Convert date columns to datetime format if they're not already
    date_columns = ['agent_join_month', 'first_policy_sold_month', 'year_month']
    for col in date_columns:
        if col in processed_df.columns and not pd.api.types.is_datetime64_any_dtype(processed_df[col]):
            processed_df[col] = pd.to_datetime(processed_df[col])
    
    # 1. Time-based features
    processed_df['agent_experience_months'] = ((processed_df['year_month'].dt.year - processed_df['agent_join_month'].dt.year) * 12 + 
                                    (processed_df['year_month'].dt.month - processed_df['agent_join_month'].dt.month))
    
    processed_df['months_since_first_policy'] = np.where(pd.notnull(processed_df['first_policy_sold_month']),
                                             ((processed_df['year_month'].dt.year - processed_df['first_policy_sold_month'].dt.year) * 12 + 
                                             (processed_df['year_month'].dt.month - processed_df['first_policy_sold_month'].dt.month)),
                                             -1)  # -1 for agents who haven't sold a policy yet
    
    # Extract date components
    processed_df['current_month'] = processed_df['year_month'].dt.month
    processed_df['current_year'] = processed_df['year_month'].dt.year
    processed_df['join_month'] = processed_df['agent_join_month'].dt.month
    processed_df['join_year'] = processed_df['agent_join_month'].dt.year
    processed_df['is_q4'] = processed_df['current_month'].isin([10, 11, 12]).astype(int)  # Q4 often has different sales patterns
    processed_df['is_q1'] = processed_df['current_month'].isin([1, 2, 3]).astype(int)
    
    # 2. Funnel conversion features
    processed_df['proposal_to_quotation_ratio'] = processed_df['unique_quotations'] / (processed_df['unique_proposal'] + 1)
    processed_df['quotation_to_customer_ratio'] = processed_df['unique_customers'] / (processed_df['unique_quotations'] + 1)
    processed_df['proposal_to_customer_ratio'] = processed_df['unique_customers'] / (processed_df['unique_proposal'] + 1)
    
    # 3. Activity trend features - Calculate slopes of activity
    processed_df['proposal_trend_1'] = processed_df['unique_proposals_last_7_days'] - processed_df['unique_proposals_last_15_days']
    processed_df['proposal_trend_2'] = processed_df['unique_proposals_last_15_days'] - processed_df['unique_proposals_last_21_days']
    processed_df['quotation_trend_1'] = processed_df['unique_quotations_last_7_days'] - processed_df['unique_quotations_last_15_days']
    processed_df['quotation_trend_2'] = processed_df['unique_quotations_last_15_days'] - processed_df['unique_quotations_last_21_days']
    processed_df['customer_trend_1'] = processed_df['unique_customers_last_7_days'] - processed_df['unique_customers_last_15_days']
    processed_df['customer_trend_2'] = processed_df['unique_customers_last_15_days'] - processed_df['unique_customers_last_21_days']
    
    # 4. Activity acceleration (second derivative)
    processed_df['proposal_acceleration'] = processed_df['proposal_trend_1'] - processed_df['proposal_trend_2']
    processed_df['quotation_acceleration'] = processed_df['quotation_trend_1'] - processed_df['quotation_trend_2']
    processed_df['customer_acceleration'] = processed_df['customer_trend_1'] - processed_df['customer_trend_2']
    
    # 5. Recent activity ratio features
    processed_df['recent_proposal_ratio'] = processed_df['unique_proposals_last_7_days'] / (processed_df['unique_proposal'] + 1)
    processed_df['recent_quotation_ratio'] = processed_df['unique_quotations_last_7_days'] / (processed_df['unique_quotations'] + 1)
    processed_df['recent_customer_ratio'] = processed_df['unique_customers_last_7_days'] / (processed_df['unique_customers'] + 1)
    
    # 6. Policy quality metrics
    if 'ANBP_value' in processed_df.columns and 'new_policy_count' in processed_df.columns:
        processed_df['avg_policy_value'] = processed_df['ANBP_value'] / (processed_df['new_policy_count'] + 0.1)
    
    if 'net_income' in processed_df.columns and 'new_policy_count' in processed_df.columns:
        processed_df['avg_policy_income'] = processed_df['net_income'] / (processed_df['new_policy_count'] + 0.1)
    
    # 7. Payment method preference
    if 'number_of_policy_holders' in processed_df.columns and 'number_of_cash_payment_policies' in processed_df.columns:
        processed_df['cash_payment_ratio'] = processed_df['number_of_cash_payment_policies'] / (processed_df['number_of_policy_holders'] + 1)
    
    # 8. Experience-adjusted performance
    # How does this agent compare to others with similar experience?
    if is_training and 'new_policy_count' in processed_df.columns:
        # Only create these features for training since they require statistics from the entire dataset
        exp_avg = processed_df.groupby('agent_experience_months')['new_policy_count'].transform('mean')
        processed_df['performance_vs_experience_peers'] = processed_df['new_policy_count'] / (exp_avg + 0.1)
    
    # 9. Activity consistency metrics
    processed_df['proposal_consistency'] = processed_df['unique_proposal'] / (processed_df['unique_proposals_last_7_days'] * 3 + 0.1)
    processed_df['quotation_consistency'] = processed_df['unique_quotations'] / (processed_df['unique_quotations_last_7_days'] * 3 + 0.1)
    processed_df['customer_consistency'] = processed_df['unique_customers'] / (processed_df['unique_customers_last_7_days'] * 3 + 0.1)
    
    # 10. Combined features
    processed_df['activity_score'] = (processed_df['unique_proposal'] + 
                                    processed_df['unique_quotations'] * 2 + 
                                    processed_df['unique_customers'] * 3)
    
    processed_df['recent_activity_score'] = (processed_df['unique_proposals_last_7_days'] + 
                                           processed_df['unique_quotations_last_7_days'] * 2 + 
                                           processed_df['unique_customers_last_7_days'] * 3)
    
    processed_df['activity_trend_score'] = (processed_df['proposal_trend_1'] + 
                                          processed_df['quotation_trend_1'] * 2 + 
                                          processed_df['customer_trend_1'] * 3)
    
    # 11. Squared features for non-linear relationships
    processed_df['agent_age_squared'] = processed_df['agent_age'] ** 2
    processed_df['experience_squared'] = processed_df['agent_experience_months'] ** 2
    
    # 12. Interaction features
    processed_df['age_experience_interaction'] = processed_df['agent_age'] * processed_df['agent_experience_months']
    processed_df['proposal_quotation_interaction'] = processed_df['unique_proposal'] * processed_df['unique_quotations']
    
    # Drop original date columns
    drop_cols = ['agent_join_month', 'first_policy_sold_month', 'year_month']
    
    # Remove new_policy_count to prevent target leakage
    if 'new_policy_count' in processed_df.columns and not is_training:
        drop_cols.append('new_policy_count')
    
    processed_df = processed_df.drop(columns=drop_cols, errors='ignore')
    
    return processed_df

# Apply feature engineering to train and test data
print("Applying feature engineering...")
train_processed = feature_engineering(train_data)
test_processed = feature_engineering(test_data, is_training=False)

# Check engineered feature set
print(f"\nNumber of features after engineering: {train_processed.shape[1]}")
print("Sample of engineered features:")
print(train_processed.columns.tolist()[:10])

#############################################
# PART 3: MODEL TRAINING AND EVALUATION
#############################################
print("\n" + "="*50)
print("MODEL TRAINING AND EVALUATION")
print("="*50)

# Define features and target
X = train_processed.drop(['row_id', 'agent_code', 'target', 'new_policy_count', 'performance_category'], axis=1, errors='ignore')
y = target

# Check for any remaining missing values
if X.isnull().sum().sum() > 0:
    print("\nHandling remaining missing values...")
    X = X.fillna(X.median())

# Split data for training and validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Feature scaling - Use RobustScaler to handle outliers better
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

print(f"\nTrain features shape: {X_train_scaled.shape}")
print(f"Validation features shape: {X_valid_scaled.shape}")

# Check class distribution
print("\nClass distribution in training set:")
print(pd.Series(y_train).value_counts(normalize=True))

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize multiple models for comparison
models = {
    'XGBoost': XGBClassifier(
        objective='binary:logistic',
        n_estimators=200,
        learning_rate=0.05,
        max_depth=5,
        min_child_weight=3,
        subsample=0.8,
        colsample_bytree=0.8,
        gamma=0.1,
        reg_alpha=0.1,
        reg_lambda=1,
        scale_pos_weight=1,
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    ),
    'LightGBM': LGBMClassifier(
        objective='binary',
        n_estimators=200,
        learning_rate=0.05,
        max_depth=5,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1,
        random_state=42
    ),
    'RandomForest': RandomForestClassifier(
        n_estimators=200,
        max_depth=8,
        min_samples_split=10,
        min_samples_leaf=4,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=5,
        min_samples_split=10,
        min_samples_leaf=4,
        subsample=0.8,
        random_state=42
    )
}

# Evaluate each model with cross-validation
print("\nModel cross-validation results:")
cv_results = {}

for name, model in models.items():
    print(f"\nEvaluating {name}...")
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='f1')
    cv_results[name] = cv_scores
    print(f"{name} - F1 Score: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    
    # Train the model on the full training set
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_valid_scaled)
    y_pred_proba = model.predict_proba(X_valid_scaled)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_valid, y_pred)
    precision = precision_score(y_valid, y_pred)
    recall = recall_score(y_valid, y_pred)
    f1 = f1_score(y_valid, y_pred)
    roc_auc = roc_auc_score(y_valid, y_pred_proba)
    
    print(f"Validation Metrics for {name}:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1 Score: {f1:.4f}")
    print(f"  ROC AUC: {roc_auc:.4f}")

# Select the best model based on cross-validation F1 score
best_model_name = max(cv_results, key=lambda k: cv_results[k].mean())
best_model = models[best_model_name]

print(f"\nBest model: {best_model_name} with average F1 score: {cv_results[best_model_name].mean():.4f}")

# Create an ensemble model (voting classifier)
print("\nCreating ensemble model...")
ensemble_model = VotingClassifier(
    estimators=[
        ('xgb', models['XGBoost']),
        ('lgbm', models['LightGBM']),
        ('rf', models['RandomForest']),
        ('gb', models['GradientBoosting'])
    ],
    voting='soft',
    weights=[2, 2, 1, 1]  # Weighting based on individual performance
)

# Train the ensemble model
ensemble_model.fit(X_train_scaled, y_train)

# Evaluate the ensemble model
ensemble_pred = ensemble_model.predict(X_valid_scaled)
ensemble_pred_proba = ensemble_model.predict_proba(X_valid_scaled)[:, 1]

# Calculate metrics for ensemble
ensemble_accuracy = accuracy_score(y_valid, ensemble_pred)
ensemble_precision = precision_score(y_valid, ensemble_pred)
ensemble_recall = recall_score(y_valid, ensemble_pred)
ensemble_f1 = f1_score(y_valid, ensemble_pred)
ensemble_roc_auc = roc_auc_score(y_valid, ensemble_pred_proba)

print("\nEnsemble Model Evaluation:")
print(f"Accuracy: {ensemble_accuracy:.4f}")
print(f"Precision: {ensemble_precision:.4f}")
print(f"Recall: {ensemble_recall:.4f}")
print(f"F1 Score: {ensemble_f1:.4f}")
print(f"ROC AUC: {ensemble_roc_auc:.4f}")

# Classification report for ensemble
print("\nClassification Report (Ensemble):")
print(classification_report(y_valid, ensemble_pred))

# Plot confusion matrix for ensemble
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_valid, ensemble_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix (Ensemble Model)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('ensemble_confusion_matrix.png')
plt.close()

# Plot ROC curve for ensemble
plt.figure(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_valid, ensemble_pred_proba)
plt.plot(fpr, tpr, lw=2, label=f'ROC curve (AUC = {ensemble_roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (Ensemble Model)')
plt.legend(loc="lower right")
plt.savefig('ensemble_roc_curve.png')
plt.close()

# Plot precision-recall curve for ensemble
plt.figure(figsize=(8, 6))
precision, recall, _ = precision_recall_curve(y_valid, ensemble_pred_proba)
avg_precision = average_precision_score(y_valid, ensemble_pred_proba)
plt.plot(recall, precision, lw=2, label=f'PR curve (AP = {avg_precision:.4f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Ensemble Model)')
plt.legend(loc="upper right")
plt.savefig('ensemble_pr_curve.png')
plt.close()

# Feature importance analysis (using XGBoost as reference)
xgb_model = models['XGBoost']
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': xgb_model.feature_importances_
})

# Sort by importance
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\nTop 15 Most Important Features:")
print(feature_importance.head(15))

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
plt.title('XGBoost Feature Importance (Top 20)')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()

#############################################
# PART 4: THRESHOLD OPTIMIZATION
#############################################
print("\n" + "="*50)
print("THRESHOLD OPTIMIZATION")
print("="*50)

# Find the optimal threshold based on F1 score
thresholds = np.linspace(0.1, 0.9, 17)
f1_scores = []
precision_scores = []
recall_scores = []

for threshold in thresholds:
    y_pred_thresh = (ensemble_pred_proba >= threshold).astype(int)
    f1_scores.append(f1_score(y_valid, y_pred_thresh))
    precision_scores.append(precision_score(y_valid, y_pred_thresh))
    recall_scores.append(recall_score(y_valid, y_pred_thresh))

# Find threshold with best F1 score
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
best_f1 = f1_scores[best_idx]

print(f"Optimal threshold: {best_threshold:.2f} with F1 score: {best_f1:.4f}")
print(f"At this threshold - Precision: {precision_scores[best_idx]:.4f}, Recall: {recall_scores[best_idx]:.4f}")

# Plot threshold optimization
plt.figure(figsize=(10, 6))
plt.plot(thresholds, f1_scores, 'o-', label='F1 Score')
plt.plot(thresholds, precision_scores, 'o-', label='Precision')
plt.plot(thresholds, recall_scores, 'o-', label='Recall')
plt.axvline(x=best_threshold, color='r', linestyle='--', label=f'Best Threshold: {best_threshold:.2f}')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Metrics vs. Classification Threshold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('threshold_optimization.png')
plt.close()

#############################################
# PART 5: FINAL PREDICTIONS AND SUBMISSION
#############################################
print("\n" + "="*50)
print("GENERATING FINAL PREDICTIONS")
print("="*50)

# Prepare test data for prediction
X_test = test_processed.drop(['row_id', 'agent_code'], axis=1, errors='ignore')

# Handle missing values in test data
if X_test.isnull().sum().sum() > 0:
    print("Handling missing values in test data...")
    X_test = X_test.fillna(X_test.median())

# Transform test data
X_test_scaled = scaler.transform(X_test)

# Make final predictions using the ensemble model
test_pred_proba = ensemble_model.predict_proba(X_test_scaled)[:, 1]
test_pred = (test_pred_proba >= best_threshold).astype(int)

# Create submission file
submission = pd.DataFrame({
    'row_id': test_data['row_id'],
    'target_column': test_pred
})

# Save submission file
submission.to_csv('submission3.csv', index=False)
print("\nFinal submission file created: submission3.csv")
print(f"Distribution of predictions: {pd.Series(test_pred).value_counts()}")

#############################################
# PART 6: AGENT INSIGHTS AND RECOMMENDATIONS
#############################################
print("\n" + "="*50)
print("AGENT INSIGHTS AND RECOMMENDATIONS")
print("="*50)

# Identify agents at risk of becoming NILL
if ensemble_pred_proba is not None:
    # Create a DataFrame with agent codes and their NILL probabilities
    agent_risk = pd.DataFrame({
        'agent_code': train_data.loc[X_valid.index, 'agent_code'],
        'nill_probability': 1 - ensemble_pred_proba,  # Probability of being NILL
        'actual_target': y_valid,
        'predicted_target': ensemble_pred
    })
    
    # Sort by risk (highest NILL probability first)
    high_risk_agents = agent_risk.sort_values('nill_probability', ascending=False).head(10)
    
    print("\nTop 10 Agents at Highest Risk of Becoming NILL:")
    print(high_risk_agents)
    
    # Group high-risk agents and analyze their characteristics
    high_risk_indices = agent_risk[agent_risk['nill_probability'] > 0.7].index
    if len(high_risk_indices) > 0:
        high_risk_profiles = X.iloc[high_risk_indices]
        low_risk_indices = agent_risk[agent_risk['nill_probability'] < 0.3].index
        low_risk_profiles = X.iloc[low_risk_indices]
        
        # Compare high vs low risk agents
        print("\nProfile Comparison - High Risk vs. Low Risk Agents:")
        comparison_cols = ['unique_proposal', 'unique_quotations', 'unique_customers', 
                          'activity_trend_score', 'agent_experience_months']
        
        high_risk_stats = high_risk_profiles[comparison_cols].mean()
        low_risk_stats = low_risk_profiles[comparison_cols].mean()
        
        comparison = pd.DataFrame({
            'High Risk': high_risk_stats,
            'Low Risk': low_risk_stats,
            'Difference %': ((low_risk_stats - high_risk_stats) / low_risk_stats * 100)
        })
        
        print(comparison)
        
        # Generate recommendations based on characteristic differences
        print("\nRecommended Interventions for High-Risk Agents:")
        print("1. Increase proposal generation activity by at least 30%")
        print("2. Focus on improving proposal-to-quotation conversion rate through training")
        print("3. Implement weekly check-ins with mentors for agents with <6 months experience")
        print("4. Provide additional sales training focused on closing techniques")
        print("5. Set up peer shadowing with medium or high performing agents")
    
    # Performance trajectory analysis
    print("\nPerformance Trajectory Analysis:")
    # Group agents by their performance trend (improving, stable, declining)
    if 'agent_code' in train_data.columns and 'year_month' in train_data.columns and 'new_policy_count' in train_data.columns:
        # Example trajectory analysis for a sample of agents
        sample_agents = train_data['agent_code'].unique()[:5]
        for agent in sample_agents:
            agent_data = train_data[train_data['agent_code'] == agent].sort_values('year_month')
            if len(agent_data) > 1:
                first_month = agent_data['new_policy_count'].iloc[0]
                last_month = agent_data['new_policy_count'].iloc[-1]
                trend = "Improving" if last_month > first_month else "Declining" if last_month < first_month else "Stable"
                print(f"Agent {agent}: {trend} trend - Started with {first_month} policies, ended with {last_month} policies")

print("\nModel Development Complete! Final submission saved as 'submission3.csv'")

INSURANCE AGENT PERFORMANCE PREDICTION MODEL
Train data shape: (15308, 23)
Test data shape: (914, 23)

EXPLORATORY DATA ANALYSIS

1. SUMMARY STATISTICS
------------------------------
                                   count                           mean  \
row_id                           15308.0                         7654.5   
agent_age                        15308.0                      40.585772   
agent_join_month                   15308  2021-05-23 01:43:05.941991168   
first_policy_sold_month            15308  2023-04-24 01:41:52.568591616   
year_month                         15308  2023-11-03 05:31:24.191272704   
unique_proposals_last_7_days     15308.0                       1.491246   
unique_proposals_last_15_days    15308.0                       2.991116   
unique_proposals_last_21_days    15308.0                      10.045728   
unique_proposal                  15308.0                      17.516527   
unique_quotations_last_7_days    15308.0                       1.98

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- customer_to_policy_rate
- performance_vs_experience_peers
- proposal_to_quotation_rate
- quotation_to_customer_rate
- unique_customers_last_7_days_ratio
- ...
